In [ ]:
%pylab notebook

# Evaluate the results of anisotropic training

## Fetch the eTRIMS data to compare against

In [ ]:
import urllib2
import zipfile
import os
import sys
import numpy as np
from glob import glob


def download(url, ):
    chunksize=2**20
    output_path = os.path.basename(url)
    with open(output_path, 'wb') as f:
        g = urllib2.urlopen(url) 
        total = int(g.info().getheaders("Content-Length")[0])
        downloaded = 0
        while True:
            chunk = g.read(chunksize)
            f.write(chunk)
            downloaded += len(chunk)
            sys.stdout.write('\r {:3.2%} Read {:>16} of {:<16} bytes'.format( downloaded /float(total), downloaded, total))
            sys.stdout.flush()
            if len(chunk) < chunksize:
                break
    sys.stdout.write('\n')
    return output_path

In [ ]:
ETRIMS_URL_PATTERN = 'http://www.ipb.uni-bonn.de/projects/etrims_db/downloads/etrims-db_{}.zip'
ETRIMS_URLS= [ETRIMS_URL_PATTERN.format(n) for n in 'v1','beta1', 'beta2']

# !mkdir -p etrims
# zipfile.ZipFile(download(ETRIMS_URLS[0])).extractall('etrims')
# zipfile.ZipFile(download(ETRIMS_URLS[1])).extractall('etrims')
# zipfile.ZipFile(download(ETRIMS_URLS[2])).extractall('etrims')
# !rm etrims-db_beta2.zip   
# !rm etrims-db_v1.zip
# !rm etrims-db_beta1.zip

In [ ]:
from glob import glob
etrims_images = glob('etrims/etrims-db_v1/images/08*/*.jpg')
etrims_labels = [fn.replace('/images/', '/annotations/').replace('.jpg','.png') for fn in etrims_images]
print len(etrims_images), "images from etrims"

In [ ]:
from pyfacades.util.metrics import Metrics

In [ ]:
!nvidia-smi

In [ ]:
CPU = True
import caffe
if CPU:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0)

In [ ]:
PROTO = 'non-bayesian-inference-net.prototxt'
#WEIGHTS = 'deploy/test_weights.caffemodel'
WEIGHTS = 'test_weights_from_peihao.caffemodel'
net = caffe.Net(PROTO, WEIGHTS, caffe.TEST)

In [ ]:
# Set the batch size to one
net.blobs['data'].reshape(1, 3, 512, 512)
net.reshape()

In [ ]:
NEG = 0
UNK = 1
POS = 2
EDG = 3

In [ ]:
import skimage.io

In [ ]:
from pyfacades.util import split_tiles, combine_tiles, softmax, channels_first, channels_last

In [ ]:
from pyfacades.rectify import Homography as AffaraRectifier

In [ ]:
from pyfacades.models.driving_12x360x480 import process_strip as segment_driving

In [ ]:
from skimage.transform import warp, ProjectiveTransform

In [ ]:
from skimage.morphology import binary_dilation, binary_erosion, disk

## Put it all together 

In [ ]:
from IPython.core.debugger import Tracer
set_trace = Tracer()

In [ ]:
%run -i color_coded_errors.py

In [ ]:
def get_metrics(f, vis=False, rectify=True):
    # Read RGB, Labels from eTRIMS
    idx = etrims_images.index(f)
    rgb = skimage.io.imread(etrims_images[idx])
    labels = skimage.io.imread(etrims_labels[idx])
    
    valid = np.ones(rgb.shape[:2])
    valid=np.pad(valid, ((0, 0), (150, 150)), mode='constant')    
    rgb=np.pad(rgb, ((0, 0), (150, 150), (0,0)), mode='constant')
    labels=np.pad(labels, ((0, 0), (150, 150), (0,0)), mode='constant')
    
    expected_windows = (labels == array([0,0,128])).all(2).astype(int)   
    se = disk(2)
    edges = binary_dilation(expected_windows, selem=se) & ~binary_erosion(expected_windows, selem=se)
    expected_windows[edges] = 2
    
    # Rectify
    presegment = segment_driving(channels_first(rgb))
    mask = binary_erosion(valid>0, selem=disk(3)) & (presegment.building() > 0.5)
    rectifier = AffaraRectifier(rgb, mask=mask)

    if rectify:    
        rectified_rgb = rectifier.rectified
        rectified_mask = rectifier.rectified_mask
        rectified_labels = warp(labels, ProjectiveTransform(rectifier.H), preserve_range=True).astype(np.uint8)
    else:
        rectified_rgb = rgb/255.
        rectified_mask = mask
        rectified_labels = labels
        
    rectified_windows = (rectified_labels == array([0,0,128])).all(2)

    # Predict / Inference
    tiles = list(split_tiles(channels_first(rectified_rgb), (512, 512)))
    prob_tiles = []
    for i, tile in enumerate(tiles):
        result = net.forward(data=np.array([tile*255]), blobs=['conv-window', 'conv-shop']) 
        prob_window = softmax(result['conv-window'][0,(0,2)])
        prob_shop = softmax(result['conv-shop'][0,(0,2)])
        
        prob_tiles.append(array([prob_window[0], prob_window[1], prob_shop[1]])) # + result['prob-shop'][0] )    
    rectified_prediction = combine_tiles(array(prob_tiles), (3,)+labels.shape[:2])
    
    # Un-Rectify for comparison
    if rectify:
        prediction =  channels_first(warp(channels_last(rectified_prediction), ProjectiveTransform(rectifier.inv_H)))
    else:
        prediction = rectified_prediction
    predicted_windows = prediction.argmax(0)
    #predicted_windows[~mask] = 0

    mf = Metrics(expected=expected_windows, 
             predicted=predicted_windows, 
             label_positive=1,
             label_negative=0,
             source=f,
             feature='window'
            )
    if vis:
        clf()
        subplot2grid((3, 5), (0,0))
        imshow(rgb)
        title('RGB')
        axis('off')
        subplot2grid((3, 5), (0,1))
        title('Labels')
        imshow(labels)
        axis('off')
        subplot2grid((3, 5), (1,0))
        title('Windows')
        imshow(expected_windows, vmin=0, vmax=2)
        axis('off')
        subplot2grid((3, 5), (1,1))
        title('Pred. Windows')
        imshow(predicted_windows, vmin=0, vmax=2)
        axis('off')
        subplot2grid((3, 5), (2,0))
        title('Rectified')
        rectifier.plot_rectified()
        title(None)
        axis('off')
        subplot2grid((3, 5), (2,1))
        title('Probs')
        imshow(prediction[1], vmin=0, vmax=1, cmap=cm.gray)
        axis('off')
        
        # Render the color coded errors in a large subplot on the right
        subplot2grid((3, 5), (0,2), colspan=3, rowspan=3)
        alpha = 0.5
        cc = color_coded_errors(expected_windows==1, predicted_windows==1, 
                                (expected_windows==2) | (predicted_windows==2))
        #set_trace()
        rgb[~cc.mask] = (1-alpha)*rgb[~cc.mask] + alpha*cc[~cc.mask]
        rgb = rgb.clip(0,255)
        imshow(rgb)
        axis('off')
        
        suptitle('A:{}, P:{}, R:{}'.format(mf.pixel_accuracy, mf.pixel_precision, mf.pixel_recall))

    return mf, channels_first(rgb), expected_windows, predicted_windows

In [ ]:
figure(figsize=(9,8))
mf, rgb, expected, predicted = get_metrics(etrims_images[12], vis=True, rectify=True)

In [ ]:
import anydbm
import json
import hashlib
import munch

In [ ]:
eval_results = anydbm.open('eval_low_rank_peihao_etrims', 'c')
checksum = hashlib.md5(open(WEIGHTS).read()).hexdigest()

In [ ]:
if 'md5' in eval_results and eval_results['md5'] == checksum:
    print "We already seem to have run evaluation..."

In [ ]:
len(etrims_images)

In [ ]:
for f in  etrims_images:
    try:
        json.loads(eval_results[f])
    except:
        print "Removing invalid", f
        del eval_results[f]

In [ ]:
!mkdir -p lr_etrims_2

In [ ]:
%pdb on
recompute = True
visualize = True

if visualize:
    fig = figure(figsize=(12,8))

total = Metrics(feature='windows')
for i, f in enumerate(etrims_images):   
    if recompute or f not in eval_results:
        mf, rgb, expected, predicted = get_metrics(f, visualize)
        eval_results[f] = json.dumps(mf.as_dict())
        if visualize:
            try:
                suptitle('{} of {}, $P$:{:.2%}, $R$:{:.2%}, $F_1$:{:.2%}, $A$:{:.2%}'.format(i, len(etrims_images), total.pixel_precision, total.pixel_recall, total.pixel_f1, total.pixel_accuracy))
            except ZeroDivisionError:
                suptitle("Not enough samples yet....")
                
            fig.canvas.draw()
            stem = os.path.splitext(os.path.basename(f))[0]
            savefig('lr_etrims_2/{}.png'.format(stem))
    else:
        mf = Metrics(**json.loads(eval_results[f]))
    #print mf, 
    total += mf
    print '\r{}  of {}: '.format(i, len(etrims_images)), total,


# Why is recall so bad?

In [ ]:
%run -i color_coded_errors.py

In [ ]:
def score(f):
    acc = Metrics(**json.loads(eval_results[f])).pixel_f1
    if isnan(acc):
        acc = 0
    return acc

In [ ]:
accs = array([score(f) for f in etrims_images])
ranking = argsort(accs)

In [ ]:
figure(figsize=(6,3))
plot(accs[ranking]);

Grab the best 9 images as examples to compare with other methods. 

In [ ]:
comparison_files = [etrims_images[ranking[-i-1]] for i in range(9)]

In [ ]:
fig = figure(figsize=(9,9))
plt.subplots_adjust(wspace=0)
for i in range(9):
    subplot(3,3,i+1)
    cached = 'separable-eTRIMS-top-{}.png'.format(i+1)
    if os.path.isfile(cached):
        err_image = imread(cached)
    else:
        err_image = render_errors(comparison_files[i], alpha=0.6);
        imsave(cached, err_image)
    imshow(err_image)
    xticks([]); yticks([]); #xlabel(cached, fontsize=8)
    fig.canvas.draw()
fig.tight_layout()
savefig('separable-eTRIMS-top-9-figure.png', dpi=400)

In the next cell I sort the images by increasing $F_1$ score. The worst images have the smallest index.

In [ ]:
accuracies = [Metrics(**json.loads(eval_results[f])).pixel_f1 for f in etrims_images]
argworst = argsort(accuracies)
worst = array(etrims_images)[argworst]
worst_indices = [etrims_images.index(k) for k in worst]

In [ ]:
figure(figsize=(8, 8))
result = get_metrics(worst[-1], vis=True)
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')
savefig('worst_example.png', dpi=300)

In [ ]:
figure(figsize=(8, 16))
result = get_metrics(worst[0], vis=True)
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title(r'Pr[window]')
savefig('worst_example.png', dpi=300)

In [ ]:
figure(figsize=(4, 8))
result = get_metrics(worst[1], vis=True)
figure(figsize=(4, 8))
result = get_metrics(worst[2], vis=True)

In [ ]:
!mkdir lr-etrims
fig = figure(figsize=(8, 8))
for i, fn in enumerate(worst):
    fig.clf()
    result = get_metrics(fn, vis=True)
    m  = result[0]
    subplot(321); title('original')
    subplot(322); title('label-colors')
    subplot(323); title('expected')
    subplot(324); title('predicted')
    subplot(325); title('rectified')
    subplot(326); title('$\Pr[\text{window}]$')
    title('A:{:.2f}, P:{:.2f}, R:{:.2f}, F:{:.2f}'.format(m.pixel_accuracy, m.pixel_precision, m.pixel_recall, m.pixel_f1))
    savefig('lr-etrims/{:03}.png'.format(i), dpi=300)
    fig.canvas.draw()

In [ ]:
from munch import Munch, munchify, toYAML

In [ ]:
all_metrics = Munch()
all_metrics.files = Munch()
total = Metrics()
for fn in etrims_images:
    stem = os.path.splitext(os.path.basename(fn))[0]
    metrics =  Metrics(**json.loads(eval_results[fn]))
    total += metrics
    all_metrics.files[stem] = munchify(metrics.as_dict())
all_metrics.total = munchify(total.as_dict())

In [ ]:
print all_metrics.total.pixel_f1
print all_metrics.total.pixel_recall
print all_metrics.total.pixel_precision

In [ ]:
with open('low_rank_etrims_attempt_one.yml', 'w') as f:
    f.write(toYAML(all_metrics))